<a href="https://colab.research.google.com/github/RushilPatel0703/ACML_Project/blob/kim/ACML_Project_Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
######## :) ########

In [2]:
####### test #######

In [3]:
# all imports

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import math
import datetime
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

Loading the data into a pandas dataframe

In [5]:
df = pd.read_csv("data.csv")
df.head()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
0,1,0.370594,0.424389,0.405750,0.601457,0.601457,0.998969,0.796887,0.808809,0.302646,...,0.716845,0.009219,0.622879,0.601453,0.827890,0.290202,0.026601,0.564050,1.0,0.016469
1,1,0.464291,0.538214,0.516730,0.610235,0.610235,0.998946,0.797380,0.809301,0.303556,...,0.795297,0.008323,0.623652,0.610237,0.839969,0.283846,0.264577,0.570175,1.0,0.020794
2,1,0.426071,0.499019,0.472295,0.601450,0.601364,0.998857,0.796403,0.808388,0.302035,...,0.774670,0.040003,0.623841,0.601449,0.836774,0.290189,0.026555,0.563706,1.0,0.016474
3,1,0.399844,0.451265,0.457733,0.583541,0.583541,0.998700,0.796967,0.808966,0.303350,...,0.739555,0.003252,0.622929,0.583538,0.834697,0.281721,0.026697,0.564663,1.0,0.023982
4,1,0.465022,0.538432,0.522298,0.598783,0.598783,0.998973,0.797366,0.809304,0.303475,...,0.795016,0.003878,0.623521,0.598782,0.839973,0.278514,0.024752,0.575617,1.0,0.035490


Viewing details of the data

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1875 entries, 0 to 1874
Data columns (total 96 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   Bankrupt?                                                 1875 non-null   int64  
 1    ROA(C) before interest and depreciation before interest  1875 non-null   float64
 2    ROA(A) before interest and % after tax                   1875 non-null   float64
 3    ROA(B) before interest and depreciation after tax        1875 non-null   float64
 4    Operating Gross Margin                                   1875 non-null   float64
 5    Realized Sales Gross Margin                              1875 non-null   float64
 6    Operating Profit Rate                                    1875 non-null   float64
 7    Pre-tax net Interest Rate                                1875 non-null   float64
 8    After-tax net Int

Checking the correlation to see the most relevent data needed

As can be seen the most useful data to use would be:


*   List item
*   List item



In [8]:
df.corr()

,Bankrupt?,ROA(C) before interest and depreciation before interest,ROA(A) before interest and % after tax,ROA(B) before interest and depreciation after tax,Operating Gross Margin,Realized Sales Gross Margin,Operating Profit Rate,Pre-tax net Interest Rate,After-tax net Interest Rate,Non-industry income and expenditure/revenue,...,Net Income to Total Assets,Total assets to GNP price,No-credit Interval,Gross Profit to Sales,Net Income to Stockholder's Equity,Liability to Equity,Degree of Financial Leverage (DFL),Interest Coverage Ratio (Interest expense to EBIT),Net Income Flag,Equity to Liability
Bankrupt?,1.000000,-0.301601,-0.310828,-0.307686,-0.095859,-0.095411,-0.036677,-0.071824,-0.097171,-0.106931,...,-0.297719,0.057724,0.010665,-0.095846,-0.148446,0.149487,0.009231,-0.002080,NaN,-0.083319
ROA(C) before interest and depreciation before interest,-0.301601,1.000000,0.952351,0.988467,0.269675,0.267972,0.317288,0.317379,0.266067,0.213646,...,0.868855,-0.160618,0.009103,0.269747,0.217163,-0.121636,-0.011878,0.017700,NaN,-0.013787
ROA(A) before interest and % after tax,-0.310828,0.952351,1.000000,0.963358,0.272278,0.270795,0.335266,0.342143,0.289045,0.241620,...,0.923150,-0.187549,0.012888,0.272334,0.251378,-0.140127,-0.005807,0.021898,NaN,0.006968
ROA(B) before interest and depreciation after tax,-0.307686,0.988467,0.963358,1.000000,0.266844,0.265256,0.322993,0.325527,0.277446,0.223201,...,0.888059,-0.178192,0.010111,0.266904,0.219478,-0.121010,-0.009491,0.018705,NaN,-0.004752
Operating Gross Margin,-0.095859,0.269675,0.272278,0.266844,1.000000,0.999643,0.263132,0.212128,0.155673,0.057631,...,0.212599,0.059571,0.006945,1.000000,0.034816,-0.027594,-0.009062,-0.001603,NaN,0.043876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Liability to Equity,0.149487,-0.121636,-0.140127,-0.121010,-0.027594,-0.027352,0.005671,-0.017320,-0.028682,-0.049999,...,-0.178496,0.060678,-0.006385,-0.027592,-0.900665,1.000000,-0.000036,0.005072,NaN,-0.149688
Degree of Financial Leverage (DFL),0.009231,-0.011878,-0.005807,-0.009491,-0.009062,-0.007853,0.006550,0.004658,0.004782,-0.000021,...,-0.004782,-0.003111,-0.018157,-0.009063,0.001617,-0.000036,1.000000,0.015266,NaN,-0.018020
Interest Coverage Ratio (Interest expense to EBIT),-0.002080,0.017700,0.021898,0.018705,-0.001603,-0.001788,0.001906,0.004703,0.005094,0.007831,...,0.017604,0.000268,0.003713,-0.001606,0.002840,0.005072,0.015266,1.000000,NaN,-0.010062
Net Income Flag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
